In [1]:
import requests
import datetime

import pandas as pd
import zipfile

import mlflow
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
#set up tracking server
TRACKING_SERVER_HOST = "34.68.82.207" #external IP reserved in GCP
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://34.68.82.207:5000'


In [4]:
data_file = "../data/processed/202304-usage.parquet"

In [5]:
mlflow.set_experiment("experiment-4")

2024/07/24 13:52:46 INFO mlflow.tracking.fluent: Experiment with name 'experiment-4' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://mlops-divvy-experiment-tracking/mlruns/4', creation_time=1721847166117, experiment_id='4', last_update_time=1721847166117, lifecycle_stage='active', name='experiment-4', tags={}>

In [7]:
with mlflow.start_run():
    #load prepared data
    df = pd.read_parquet(data_file)
    mlflow.log_param("data_file", data_file)

    features = df[['station_name', 'hour', 'day_of_week']]
    target = df['net_usage']

    num_features= ['hour']
    cat_features = ['station_name', 'day_of_week']

    split_params = {"test_size": 0.2, "random_state": 42}
    X_train, X_test, y_train, y_test = train_test_split(features, target, **split_params)

    Standard_Scaler = StandardScaler()
    num_scaled_train = pd.DataFrame(Standard_Scaler.fit_transform(X_train[num_features]), columns=['hour_scaled'])
    num_scaled_test = pd.DataFrame(Standard_Scaler.transform(X_test[num_features]), columns=['hour_scaled'])

    num_scaled_train.index = X_train.index
    num_scaled_test.index = X_test.index

    ohe = OneHotEncoder(handle_unknown='ignore')
    ohe_cols_train = ohe.fit_transform(X_train[cat_features])
    ohe_cols_test = ohe.transform(X_test[cat_features])

    ohe_cols_train = pd.DataFrame(ohe.fit_transform(X_train[cat_features]).toarray(), columns = ohe.get_feature_names_out(cat_features))
    ohe_cols_test = pd.DataFrame(ohe.transform(X_test[cat_features]).toarray(), columns = ohe.get_feature_names_out(cat_features))

    ohe_cols_train.index = X_train.index
    ohe_cols_test.index = X_test.index

    transformed_X_train = pd.concat([num_scaled_train, ohe_cols_train], axis=1)
    transformed_X_test = pd.concat([num_scaled_test, ohe_cols_test], axis=1)

    params = {"n_estimators": 10, "random_state": 42}
    mlflow.log_params(params)
    rf = RandomForestRegressor(**params).fit(transformed_X_train, y_train)
    
    y_pred = rf.predict(transformed_X_test)
    mlflow.log_metric("mse", mean_squared_error(y_test, y_pred))

    mlflow.sklearn.log_model(rf, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

mlflow.search_experiments()

NameError: name 'params' is not defined